<a href="https://colab.research.google.com/github/sheethalkaran/FitPulse-Health-Anomaly-Detection-from-Fitness-Devices/blob/main/Milestone4/dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
!pip install streamlit prophet pyngrok pandas matplotlib --quiet

In [35]:
from pyngrok import ngrok
ngrok.kill()
ngrok.set_auth_token("YOUR_NGROK_ID")

In [33]:
%%writefile app.py
import streamlit as st
import pandas as pd
import plotly.express as px
from prophet import Prophet

st.set_page_config(page_title="FitPulse Health Dashboard", page_icon="❤️", layout="wide")

st.markdown("""
<style>
body { background:#020617; }

.block-container {
    padding-top: 1.2rem !important;
    padding-bottom: 1.2rem !important;
}

[data-testid="stVerticalBlock"] {
    gap: 0.6rem !important;
}

[data-testid="stSidebar"] { background:#020617; }
[data-testid="stSidebar"] * { color:#e5e7eb !important; }

h1 {
    color:#e5e7eb;
    margin-top: 0.6rem !important;
    margin-bottom: 1rem !important;
}

.metric-card {
    background:linear-gradient(145deg,#020617,#0f172a);
    border-radius:18px;
    padding:18px;
    text-align:center;
    box-shadow:0px 10px 25px rgba(0,0,0,0.45);
    height:110px;
}

.metric-value {
    font-size:30px;
    font-weight:800;
    color:#38bdf8;
}

.metric-label {
    color:#94a3b8;
    font-size:12.5px;
}

.stPlotlyChart iframe {
    border-radius:18px !important;
    overflow:hidden !important;
}

.stPlotlyChart {
    margin-top: 0.6rem !important;
    margin-bottom: 0.8rem !important;
}

[data-testid="stDownloadButton"] button {
    background: linear-gradient(145deg, #020617, #0f172a) !important;
    color: #38bdf8 !important;
    border-radius: 18px !important;
    padding: 10px 22px !important;
    border: none !important;
    box-shadow: 0px 10px 25px rgba(0,0,0,0.45) !important;
    font-size: 15px !important;
    font-weight: 600 !important;
}

[data-testid="stDownloadButton"] button:hover {
    color: #7dd3fc !important;
    background: linear-gradient(145deg, #020617, #111827) !important;
}

[data-testid="stFileUploader"] section { background:#f8fafc !important; }
[data-testid="stFileUploader"] button,
[data-testid="stFileUploader"] label,
[data-testid="stFileUploader"] span { color:black !important; }

[data-testid="stSelectbox"] div,
[data-testid="stDateInput"] input,
ul[role="listbox"] * { color:black !important; }

ul[role="listbox"] { background:#f8fafc !important; }

input { color:black !important; }

</style>
""", unsafe_allow_html=True)

st.title("🫀 FitPulse – Health Anomaly Detection Dashboard")

st.sidebar.header("📂 Dataset Upload & Filters")

file = st.sidebar.file_uploader("Upload Fitness Data (CSV / JSON)", type=["csv","json"])

if file:
    df = pd.read_json(file, lines=True) if file.name.endswith(".json") else pd.read_csv(file)

    if not {"DateTime","Id","HeartRate","Steps","TotalMinutesAsleep"}.issubset(df.columns):
        st.error("Dataset missing required columns")
        st.stop()

    df["DateTime"] = pd.to_datetime(df["DateTime"], errors="coerce").dt.tz_localize(None)
    df["HeartRate"] = pd.to_numeric(df["HeartRate"], errors="coerce")
    df["Steps"] = pd.to_numeric(df["Steps"], errors="coerce")
    df["TotalMinutesAsleep"] = pd.to_numeric(df["TotalMinutesAsleep"], errors="coerce")

    df["HeartRate"].fillna(df["HeartRate"].median(), inplace=True)
    df["Steps"].fillna(0, inplace=True)
    df["TotalMinutesAsleep"].fillna(df["TotalMinutesAsleep"].median(), inplace=True)


    df = df.dropna(subset=["DateTime"]).sort_values("DateTime")

    user_id = st.sidebar.selectbox("Select User", sorted(df["Id"].unique()))
    metric = st.sidebar.selectbox("Select Metric", ["HeartRate","Steps","TotalMinutesAsleep"])

    date_range = st.sidebar.date_input(
    "Select Date Range",
    [df["DateTime"].min().date(), df["DateTime"].max().date()]
    )

    if not isinstance(date_range, (list, tuple)) or len(date_range) != 2:
        st.error("Please select both start and end dates.")
        st.stop()

    start_date, end_date = date_range

    user_df = df[
        (df["Id"] == user_id) &
        (df["DateTime"].dt.date >= start_date) &
        (df["DateTime"].dt.date <= end_date)
    ]

    if len(user_df) < 5:
        st.warning("Not enough data points")
        st.stop()

    model_df = user_df[["DateTime",metric]].rename(
        columns={"DateTime":"ds",metric:"y"}
    ).dropna().sort_values("ds")

    model = Prophet(daily_seasonality=True, weekly_seasonality=False, yearly_seasonality=False)
    model.fit(model_df)

    forecast = model.predict(model_df[["ds"]])

    model_df["predicted"] = forecast["yhat"].values[:len(model_df)]
    model_df["lower"] = forecast["yhat_lower"].values[:len(model_df)]
    model_df["upper"] = forecast["yhat_upper"].values[:len(model_df)]


    model_df["residual"] = model_df["y"] - model_df["predicted"]

    std = model_df["residual"].std()
    threshold = 3 * std if std > 0 else 0.1

    model_df["anomaly"] = (
        (abs(model_df["residual"]) > threshold) |
        (model_df["y"] < model_df["lower"] - std) |
        (model_df["y"] > model_df["upper"] + std)
    )

    if metric == "TotalMinutesAsleep":
      model_df["anomaly"] = model_df["anomaly"] | (model_df["y"] == 0)

    c1,c2,c3 = st.columns(3)

    c1.markdown(f"""
    <div class="metric-card">
        <div class="metric-value">{len(model_df)}</div>
        <div class="metric-label">Total Records</div>
    </div>
    """, unsafe_allow_html=True)

    c2.markdown(f"""
    <div class="metric-card">
        <div class="metric-value">{model_df['anomaly'].sum()}</div>
        <div class="metric-label">Anomalies Detected</div>
    </div>
    """, unsafe_allow_html=True)

    c3.markdown(f"""
    <div class="metric-card">
        <div class="metric-value">{model_df['y'].mean():.2f}</div>
        <div class="metric-label">Average {metric}</div>
    </div>
    """, unsafe_allow_html=True)

    fig = px.line(
    model_df,
    x="ds",
    y="y",
    title=f"{metric} Trend",
    template="plotly_dark",
    height=320
    )

    fig.add_scatter(
        x=model_df[~model_df["anomaly"]]["ds"],
        y=model_df[~model_df["anomaly"]]["y"],
        mode="markers",
        marker=dict(color="limegreen", size=6),
        name="Normal"
    )

    fig.add_scatter(
        x=model_df[model_df["anomaly"]]["ds"],
        y=model_df[model_df["anomaly"]]["y"],
        mode="markers",
        marker=dict(color="red", size=7),
        name="Anomaly"
    )

    fig.update_layout(
    title=dict(
        text=f"{metric} Trend",
        font=dict(color="white", size=13)
    ),
    legend=dict(
        font=dict(color="white", size=12),
        bgcolor="rgba(0,0,0,0)"
    ),
    xaxis=dict(
        title_font=dict(color="white", size=12),
        tickfont=dict(color="white", size=11)
    ),
    yaxis=dict(
        title_font=dict(color="white", size=12),
        tickfont=dict(color="white", size=11)
    ),
    margin=dict(l=30, r=30, t=45, b=35),
    plot_bgcolor="#020617",
    paper_bgcolor="#020617"
)

    st.plotly_chart(fig, use_container_width=True)

    report = model_df[
        ["ds","y","predicted","residual","anomaly"]
    ].rename(columns={
        "ds": "Timestamp",
        "y": metric,
        "predicted": "Expected",
        "residual": "Deviation",
        "anomaly": "Anomaly"
    })

    st.download_button(
        "Download CSV Report",
        report.to_csv(index=False),
        "fitpulse_anomaly_report.csv",
        "text/csv"
    )

else:
    st.info("Upload a fitness dataset to begin analysis")

Overwriting app.py


In [34]:
!streamlit run app.py &>/content/streamlit.log &

In [ ]:
public_url = ngrok.connect(8501)
print("Dashboard URL:", public_url)